## 異なるモデルの比較

このノートブックでは、これまで利用してきたelyza/Llama-3-ELYZA-JP-8Bと並行して別のモデルであるMistral-7Bを使用し、その挙動を観察します。
Mistral-7Bは日本語だけでなく多言語対応したモデルです。両者のアウトプットの違いを確認してみましょう。

### 必要なライブラリとインポート

Labの指示に従って適切なワークベンチイメージを選択して起動した場合、必要なすべてのライブラリがすでにインストールされているはずです。もしインストールされていない場合は、次のセルの最初の行のコメントを外して正しいパッケージをすべてインストールしてください。その後、必要なライブラリをインポートします。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # 正しいワークベンチイメージを選択していない場合のみコメントを外してください

import json
import os
import httpx
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)

### Langchainパイプライン

Langchainを使用して、パイプラインを定義します。

In [ ]:
# LLM推論APIのURL
inference_server_url = "_INFERENCE_URL_LLM_"

# LLMの定義
llm = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base= f"{inference_server_url}/v1",
    model_name="elyza",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    async_client=httpx.AsyncClient(verify=False),
    http_client=httpx.Client(verify=False)
)

In [ ]:
# Mistral LLM推論サーバーURL
inference_server_url_mistral = "http://llm.ic-shared-llm.svc.cluster.local:8000"

# LLMの定義
llm_mistral = VLLMOpenAI(
    openai_api_key="EMPTY",
    openai_api_base= f"{inference_server_url_mistral}/v1",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

それぞれのモデルに合わせた**テンプレート**を作成します。

In [ ]:
system_template_string = """
あなたは、親切で、礼儀正しく、正直なアシスタントです。
常に気配りと尊重をもって接し、真摯にサポートします。できる限り有用な返答を提供しますが、安全を確保します。
有害で、倫理に反する、偏見のある、または否定的な内容は避けます。返答が公正でポジティブなものであることを確認します。
"""

user_template_string = """
与えられた文章の内容をもとに、与えられた質問に答えてください。

### 文章:
{text}

### 質問:
{query}

### 回答:
"""

system_template = SystemMessagePromptTemplate.from_template(system_template_string)
user_template = HumanMessagePromptTemplate.from_template(user_template_string)

PROMPT = ChatPromptTemplate.from_messages([system_template, user_template])

In [ ]:
template="""<s>[INST]
あなたは、親切で、礼儀正しく、正直なアシスタントです。
常に気配りと尊重をもって接し、真摯にサポートします。できる限り有用な返答を提供しますが、安全を確保します。
有害で、倫理に反する、偏見のある、または否定的な内容は避けます。返答が公正でポジティブなものであることを確認します。

### テキスト:
{text}

### 質問:
{query}

### 回答:
[/INST]
"""
PROMPT_MISTRAL = PromptTemplate(input_variables=["input"], template=template)

2つの**会話**オブジェクトを作成し、それぞれのモデルにクエリを投げる準備が行います。

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )
conversation_mistral = LLMChain(llm=llm_mistral,
                        prompt=PROMPT_MISTRAL,
                        verbose=False
                        )

モデルにクエリを投げる準備が整いました！

この例では、1つの請求文章を対象にクエリを実行しどのような結果が得られるかを見てみます。もちろん、他の請求文章で試してみても大丈夫です。

In [ ]:
filename = 'claims/claim1.json'

# Opening JSON file
claims = {}
with open(filename, 'r') as file:
    data = json.load(file)
claims[filename] = data

# Analyze the claim
print(f"***************************")
print(f"* 請求: {filename}")
print(f"***************************")
print("元の文章:")
print("-----------------")
print(f"件名: {claims[filename]['subject']}\n内容:\n{claims[filename]['content']}\n\n")
print('Elyzaによる分析:')
print("--------")
text_input = f"件名: {claims[filename]['subject']}\n内容:\n{claims[filename]['content']}"
sentiment_query = "この請求の文章から読み取れる感情はどのようなものですか？「肯定的」、「否定的」、「どちらでもない」から1つだけ選んで答え、その理由もあわせて説明してください。"
location_query = "この請求に関連する出来事はどこで起こりましたか？出来事の発生した場所について、市区町村や通りの名前などを答えて下さい。"
time_query = "この請求に関連する出来事はいつ起こりましたか？日付と、時刻あるいは時間帯を一言で答えて下さい。"
print(f"- 送信者の感情: ")
conversation.predict(text=text_input, query=sentiment_query);
print("\n- 発生場所: ")
conversation.predict(text=text_input, query=location_query);
print("\n- 発生日時: ")
conversation.predict(text=text_input, query=time_query);
print("\n\n                          ----====----\n")
print('Mistralによる分析:')
print("--------")
text_input = f"件名: {claims[filename]['subject']}\n内容:\n{claims[filename]['content']}"
sentiment_query = "この請求の文章から読み取れる感情はどのようなものですか？「肯定的」、「否定的」、「どちらでもない」から1つだけ選んで答え、その理由もあわせて説明してください。"
location_query = "この請求に関連する出来事はどこで起こりましたか？出来事の発生した場所について、市区町村や通りの名前などを答えて下さい。"
time_query = "この請求に関連する出来事はいつ起こりましたか？日付と、時刻あるいは時間帯を一言で答えて下さい。"
print(f"- 送信者の感情: ")
conversation_mistral.predict(text=text_input, query=sentiment_query);
print("\n- 発生場所: ")
conversation_mistral.predict(text=text_input, query=location_query);
print("\n- 発生日時: ")
conversation_mistral.predict(text=text_input, query=time_query);
print("\n\n                          ----====----\n")

両者の出力はどちらも正しいですが、日本語に特化したチューニングが行われたElyzaの方がより自然な回答となっています。

LLMを扱う際の技術は、求めるパフォーマンスと精度の間、ならびにそれに伴うリソースやコストとのバランスを見つけることです。

そのため、データが変化したり、モデルが進化したりする際に、常に期待通りの挙動が得られるようにするための信頼性チェックを行うことが重要となります。